In [1]:
import os
import sys 
main_dir=os.getcwd()
module_dir=(main_dir+"\\modules")
if module_dir not in sys.path:
    sys.path.append(module_dir)
import traitlets
import ipywidgets as wg
import re
#import log_select
import log_decode
import hbm_mapping
import HBM_lib
import pandas as pd
from IPython.display import clear_output
from IPython.display import display
from tkinter import Tk, filedialog
#import warnings
#warnings.filterwarnings('ignore')




##################################################################################################################
################################log file select, file info & decode btn###########################################
select_file_btn=wg.Button(description='Select File', button_style='info')
decode_btn=wg.Button(description='Decode', button_style='primary')
logfile_Text_layout=wg.Layout(display='flex',flex_flow='row',align_items='center',border='',width='600px', height='',justify_content='center')
logfile_Text=wg.Textarea(value='',placeholder='',description='Log file',disabled=True, layout=logfile_Text_layout )
#================================log file select, file info & decode btn Hbox====================================#
log_select_hbox_layout=wg.Layout(display='flex',flex_flow='row',align_items='center',border='',width='930px',justify_content='space-between')
log_select_hbox=wg.HBox([select_file_btn, logfile_Text,decode_btn],layout=log_select_hbox_layout)
#=======================================high_level_summary_output Hbox ==============================================#
high_level_summary_output_hbox_layout=wg.Layout(display='flex',flex_flow='column',align_items='center',border='',width='930px',justify_content='space-between')
high_level_summary_output_hbox=wg.HBox([],layout=high_level_summary_output_hbox_layout)



#=====================================SID select check box=======================================================#
CDC_SID_layout=wg.Layout(display='flex',flex_flow='row',align_items='center',border='',width='170px',justify_content='center')
CDC_SID0=wg.Checkbox(value=False,description='CDC-SID0 Failed',disabled=False,indent=False,layout=CDC_SID_layout)
CDC_SID1=wg.Checkbox(value=False,description='CDC-SID1 Failed',disabled=False,indent=False,layout=CDC_SID_layout)
CDC_SID2=wg.Checkbox(value=False,description='CDC-SID2 Failed',disabled=False,indent=False,layout=CDC_SID_layout)
CDC_Unknown=wg.Checkbox(value=False,description='CDC Fail Unknown',disabled=False,indent=False,layout=CDC_SID_layout)

def CDC_Unknown_observer(sender):
    CDC_SID0.value=False
    CDC_SID1.value=False
    CDC_SID2.value=False
CDC_Unknown.observe(CDC_Unknown_observer, names=['value'])

def CDC_SID_observer(sender):
    CDC_Unknown.value=False
CDC_SID0.observe(CDC_SID_observer, names=['value'])  
CDC_SID1.observe(CDC_SID_observer, names=['value'])   
CDC_SID2.observe(CDC_SID_observer, names=['value'])

CDC_select_layout=wg.Layout(display='flex',flex_flow='row',align_items='center',border='',width='930px',justify_content='space-between')
CDC_select_hbox=wg.HBox([CDC_SID0, CDC_SID1,CDC_SID2,CDC_Unknown],layout=CDC_select_layout)
CDC_list=[]
#=============================================== hbm map Vbox ===================================================#
hbm_map_vbox_layout=wg.Layout(display='flex',flex_flow='column',align_items='center',border='',width='100%',justify_content='space-between')
hbm_map_vbox=wg.VBox([],layout=hbm_map_vbox_layout)
################################################## Main vertical box #################################################
Main_vbox_layout=wg.Layout(display='flex',flex_flow='column',align_items='center',border='solid',width='100%',justify_content='space-between')
Main_vbox=wg.VBox([CDC_select_hbox,log_select_hbox],layout=Main_vbox_layout)
############################################################################################################
display(Main_vbox)
display(HBM_lib.HBM_lib.lib_frame_vbox)
#============CSV For SAVING DF =============
DF_CSV=HBM_lib.HBM_lib.lib.copy()
#============CSV For SAVING DF =============
#################################################################################################
log_file=''
def run_select_file(b):
    global logfile_Text
    global log_file
    global CDC_SID0
    global CDC_SID1
    global CDC_SID2
    global CDC_Unknown
    # Create Tk root
    root = Tk()
    # Hide the main window
    root.withdraw()
    # Raise the root to the top of all windows.
    root.call('wm', 'attributes', '.', '-topmost', True)
    # List of selected fileswill be set to b.value
    b.files = filedialog.askopenfilename(multiple=True)
    #b.button_style='success'
    #b.description = "Files Selected"
    try:
        log_dir_name_list=b.files[0].split('/')
        log_name=log_dir_name_list[len(log_dir_name_list)-1]
        log_file=b.files[0].replace('/','\\')
        logfile_Text.value=log_file
        CDC_SID0.value=False
        CDC_SID1.value=False
        CDC_SID2.value=False
        CDC_Unknown.value=False
    except:
        pass
select_file_btn.on_click(run_select_file)    
#################################################################################################
def run_decode(b):
    global Main_vbox
    global hbm_map_vbox
    global log_file
    global logfile_Text
    global CDC_list
    global DF_CSV
    
    #Main_vbox.children=[log_select_hbox]
    
    if log_file == '':
        logfile_Text.value='Please select log file'
    else:
        CDC_list=[CDC_SID0.value,CDC_SID1.value,CDC_SID2.value,CDC_Unknown.value]
        globals()['Decode']=log_decode.HBM3_12H_ATE_log_decode(log_file)
        b.description='Decoded'
        b.button_style='success'
        
        high_level_summary_output = wg.Output()
        with high_level_summary_output:
            display(Decode.log_bank_review_display2)
        high_level_summary_output_hbox.children=[high_level_summary_output]
        hbm_map_list=[]    
        for i in range(len(Decode.log_core_review)):
            HBM=Decode.log_core_review.loc[i,'HBM']
            Core=Decode.log_core_review.loc[i,'Core']
            Channel=Decode.log_core_review.loc[i,'Channel']
            Bank=Decode.log_core_review.loc[i,'Bank']
            Failed_Test=Decode.log_core_review.loc[i,'Test_failed']
            Fail_mode=Decode.log_core_review.loc[i,'Bank_assess']
            globals()['Map'+str(i)]=hbm_mapping.HBM_map(HBM,Core,Channel,Bank,Failed_Test,Fail_mode,CDC_list) ##add cdc
            hbm_map_list.append(eval('Map'+str(i)).hbm_vbox)
        hbm_map_vbox.children=hbm_map_list
        Main_vbox.children=[CDC_select_hbox,log_select_hbox,high_level_summary_output_hbox,hbm_map_vbox]
        clear_output()
        #### lib filtering for related lib ###
        hbm_list=list(Decode.log_core_review['HBM'])
        core_lis=list(Decode.log_core_review['Core'])
        ch_list=[]
        for i in list(Decode.log_core_review['Channel']):
            ch_split=i.split(',')
            for j in ch_split:
                ch=j.split(',')
                ch_list.append(ch[0])
        ch_list=list(set(ch_list))
        bank_list=[]
        for i in list(Decode.log_core_review['Bank']):
            bank_split=i.split(',')
            for j in bank_split:
                bank=j.split(',')
                bank_list.append(bank[0])
        bank_list=list(set(bank_list))
        filtered_df=HBM_lib.HBM_lib.lib[HBM_lib.HBM_lib.lib['HBM'].isin(hbm_list)]
        filtered_df=filtered_df[filtered_df['Core'].isin(core_lis)]
        filtered_df=filtered_df[filtered_df['Channel'].isin(ch_list)]
        filtered_df=filtered_df[filtered_df['Bank'].isin(bank_list)]
        filtered_df=filtered_df.reset_index(drop=True)
        #============CSV For SAVING DF =============
        DF_CSV=filtered_df.copy()
        #============CSV For SAVING DF =============
        decode_filter_output = wg.Output()
        with decode_filter_output:
            display(filtered_df)
        HBM_lib.HBM_lib.Lib_df_hbox.children=[decode_filter_output]
        #### lib filtering for related lib ###
        #display(Map0.hbm_sip_pic,Map0.hbm_core_pic,Map0.hbm_bank_pic,Main_vbox)
        display(Main_vbox)
        display(HBM_lib.HBM_lib.lib_frame_vbox)
decode_btn.on_click(run_decode)    
########################################################################################################

def search(b):
    global DF_CSV
    HBM_value_list=list(HBM_lib.HBM_lib.HBM_select_box.value)
    Core_value_list=list(HBM_lib.HBM_lib.Core_select_box.value)
    Ch_values=[x for x in list(HBM_lib.HBM_lib.Ch_select_box.value) if re.findall('[A-Z]',x)]
    Ch_value_list=[]
    for i in Ch_values:
        ch_str=i+str('0')
        Ch_value_list.append(ch_str)
        ch_str=i+str('1')
        Ch_value_list.append(ch_str)
    Bank_value_list=list(HBM_lib.HBM_lib.Bank_select_box.value)
    Defect_type_value_list=list(HBM_lib.HBM_lib.Defect_type_select_box.value)
    Defect_location_value_list=list(HBM_lib.HBM_lib.Defect_location_select_box.value)
    
    if len(HBM_value_list) != 0:
        filtered_df=HBM_lib.HBM_lib.lib[HBM_lib.HBM_lib.lib['HBM'].isin(HBM_value_list)]
    else:
        filtered_df=HBM_lib.HBM_lib.lib
    if len(Core_value_list) != 0:
        filtered_df=filtered_df[filtered_df['Core'].isin(Core_value_list)]
    else:
        filtered_df=filtered_df
    if len(Ch_value_list) != 0:
        filtered_df=filtered_df[filtered_df['Channel'].isin(Ch_value_list)]
    else:
        filtered_df=filtered_df  
    if len(Bank_value_list) != 0:
        filtered_df=filtered_df[filtered_df['Bank'].isin(Bank_value_list)]
    else:
        filtered_df=filtered_df
    if len(Defect_type_value_list) != 0:
        filtered_df=filtered_df[filtered_df['Defect_type_PFA'].isin(Defect_type_value_list)]
    else:
        filtered_df=filtered_df
    if len(Defect_location_value_list) != 0:
        filtered_df=filtered_df[filtered_df['Defect_location_PFA'].isin(Defect_location_value_list)]
    else:
        filtered_df=filtered_df
    #============CSV For SAVING DF =============
    DF_CSV=filtered_df.copy()
    #============CSV For SAVING DF =============
    lib_filter_output = wg.Output()
    with lib_filter_output:
        display(filtered_df)
    HBM_lib.HBM_lib.Lib_df_hbox.children=[lib_filter_output]
    clear_output()
    try:
        Map0.hbm_sip_pic
        display(Map0.hbm_sip_pic,Map0.hbm_core_pic,Map0.hbm_bank_pic,Main_vbox)
        display(HBM_lib.HBM_lib.lib_frame_vbox)
    except NameError:
        display(Main_vbox)
        display(HBM_lib.HBM_lib.lib_frame_vbox)
HBM_lib.HBM_lib.Search_btn.on_click(search)    

########################################################################################################

def unselect(b):
    global DF_CSV
    HBM_lib.HBM_lib.HBM_select_box.value=()
    HBM_lib.HBM_lib.Core_select_box.value=()
    HBM_lib.HBM_lib.Ch_select_box.value=()
    HBM_lib.HBM_lib.Bank_select_box.value=()
    HBM_lib.HBM_lib.Defect_type_select_box.value=()
    HBM_lib.HBM_lib.Defect_location_select_box.value=() 
    HBM_lib.HBM_lib.lib_output = wg.Output()
    with HBM_lib.HBM_lib.lib_output:
        display(HBM_lib.HBM_lib.lib)
    HBM_lib.HBM_lib.Lib_df_hbox.children=[HBM_lib.HBM_lib.lib_output]
    #============CSV For SAVING DF =============
    DF_CSV=HBM_lib.HBM_lib.lib.copy()
    #============CSV For SAVING DF =============
    clear_output()
    try:
        Map0.hbm_sip_pic
        display(Map0.hbm_sip_pic,Map0.hbm_core_pic,Map0.hbm_bank_pic,Main_vbox)
        display(HBM_lib.HBM_lib.lib_frame_vbox)
    except NameError:
        display(Main_vbox)
        display(HBM_lib.HBM_lib.lib_frame_vbox)
HBM_lib.HBM_lib.open_Library_btn.on_click(unselect)    


########################################################################################################
def Save_lib(b):
    global DF_CSV
    # Create Tk root
    root = Tk()
    # Hide the main window
    root.withdraw()
    # Raise the root to the top of all windows.
    root.call('wm', 'attributes', '.', '-topmost', True)
    # List of selected fileswill be set to b.value
    b.file_path = filedialog.asksaveasfilename()
    try:
    #b.button_style='success'
    #b.description = "LIB saved"
        file_path=b.file_path.replace('/','\\')
        DF_CSV.to_csv(file_path)
    except:
        pass

HBM_lib.HBM_lib.save_lib_btn.on_click(Save_lib)

########################################################################################################





######from IPython.display import HTML
######
######HTML('''<script>
######code_show=true; 
######function code_toggle() {
###### if (code_show){
###### $('div.input').hide();
###### } else {
###### $('div.input').show();
###### }
###### code_show = !code_show
######} 
######$( document ).ready(code_toggle);
######</script>
######<form action="javascript:code_toggle()"><input type="submit" 
######value="Click here to toggle on/off the raw code."></form>''')
######
######